In [1]:
import pandas as pd


In [2]:
df = pd.read_json("./rege101.json")

ValueError: Expected object or value

In [4]:
import json
with open("./regex101.json", "r") as reader:
    data = json.load(reader)



In [8]:
df = pd.DataFrame.from_records(data)

In [14]:
df_cleaned = df.drop_duplicates()

In [16]:
df_cleaned[df_cleaned.flavor == "python"]

,title,description,dateCreated,dateModified,author,flavor,version,permalinkFragment,upvotes,downvotes,score,userVote,isFavorite,isStarred
13,IOS3166 Country Code Identification REGEX,IOS3166 Country Code Identification REGEX,2015-06-07T04:48:58.000Z,2015-06-07T04:48:58.000Z,theitgeek@recu.org.uk,python,1,wV8dJ2,6,0,0.609657,None,0,0
15,Find Reddit Threads,Simply searches for the thread-id in any kind ...,2015-03-12T09:23:09.000Z,2015-03-12T09:23:09.000Z,Mio,python,1,tY0dS0,12,2,0.600581,None,0,0
16,entity,,2015-11-20T01:12:02.000Z,2015-11-20T01:12:02.000Z,,python,1,sE5tS4,12,2,0.600581,None,0,0
18,Distinguish torrent files (series vs movies),A neat regex for finding out whether a given t...,2014-06-26T09:59:45.000Z,2023-07-20T15:08:14.000Z,Firas Dib,python,3,yP4bY4,178,97,0.589123,None,0,0
33,Form Tag,Form Tag,2016-02-26T06:49:56.000Z,2016-02-26T06:49:56.000Z,Jay Patel,python,1,pB8bN3,5,0,0.565509,None,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17402,选择题题目与选项匹配,对有ABCD四个选项的选择题题目进行匹配,2023-07-07T00:59:51.000Z,2023-07-07T00:59:51.000Z,Wang Miao,python,1,j9g0yE,0,0,0.000000,None,0,0
17407,Fate Stay/Night (PC/JP) [Py],Attempting to parse Fate/Stay Night scripts ni...,2023-07-08T02:50:46.000Z,2023-07-08T02:50:46.000Z,,python,1,F8LC4p,0,0,0.000000,None,0,0
17408,Fate stay/night [HuneX/PSV],Fate stay/night PS Vita version parsing,2023-07-08T02:52:53.000Z,2023-07-08T05:58:56.000Z,,python,3,8BGCdu,0,0,0.000000,None,0,0
17412,r/ProgrammerHumor comment regex,A regex that matches comments that adhere to t...,2023-07-09T17:24:27.000Z,2023-07-09T17:24:27.000Z,,python,1,ah8XM8,0,0,0.000000,None,0,0


In [29]:
import os
import time
import requests
import tqdm


def download_details(regex_ids):
    data = []
    if "regex101_detailed.json" in os.listdir():
        with open("regex101_detailed.json", "r") as reader:
            data = json.load(reader)

    downloaded_ids = []
    backoff = 1.5
    timeout = 120

    downloaded_df = pd.DataFrame.from_records(data)
    downloaded_ids = set(downloaded_df.permalinkFragment.to_list())
    pending_ids = list(set(regex_ids).difference(downloaded_ids))

    url_format = "https://regex101.com/api/library/details/{re_id}"

    # Flitrar las ids que estan ya en detailed antes de empezar a procesarlo

    try:
        downloaded_ids = []
        for re_id in tqdm.tqdm(pending_ids):
            res = requests.get(url_format.format(re_id=re_id))

            while res.status_code != 200:
                res = requests.get(url_format.format(re_id=re_id))
                timeout = backoff * timeout
                time.sleep(timeout)

            timeout = 120

            data.append(res.json())
            downloaded_ids.append(re_id)
            time.sleep(2)
    except Exception:
        import traceback
        traceback.print_exc()
    finally:
        with open("./regex101_detailed.json", "w") as writer:
            writer.write(json.dumps(data))
    
    return downloaded_ids

In [30]:
downloaded_ids = download_details(df_cleaned.permalinkFragment.to_list())
# 1715

  0%|          | 0/14428 [00:00<?, ?it/s]

  0%|          | 59/14428 [02:09<8:45:15,  2.19s/it]

In [24]:
len(downloaded_ids)

NameError: name 'downloaded_ids' is not defined

In [21]:
df_cleaned.shape

(17400, 14)